# 建立User類別
擁有 buy_stock、sell_stock


In [1]:
import pandas as pd
import yfinance as yf
import datetime as dt
#建立user類別
#屬性：使用者名稱、交割餘額、股票紀錄、手續費、持有股票
#方法：買股票、賣股票、更新持有股票
class User():

    def __init__(self,name, balance=1000000):
        self.name = name #使用者名稱
        self.balance = balance #交割餘額
        self.handling_fee = 0.001425 #手續費
        self.auto = False #是否自動交易
        
        try:
            self.__load_from_csv()
        except:
            self.position = pd.DataFrame(columns = ['stock_name','amount','price','avg_price']) #持有股票
            self.record = pd.DataFrame(columns = ['date','stock_name','amount','price','avg_price']) #股票紀錄

    def __save_to_csv(self, r_filename='record.csv',p_filename='position.csv'):
        self.record.to_csv(self.name+r_filename, index=False)
        self.position.to_csv(self.name+p_filename,index=False)

    def __load_from_csv(self, r_filename='record.csv',p_filename='position.csv'):
        self.record = pd.read_csv(self.name+r_filename)
        self.position= pd.read_csv(self.name+p_filename)

    def __update(self): # private 禁止外部使用，以防重複計算
        
        '''
        Update the position based on the records.
        '''
        
        if self.auto and not self.record.empty:
            # Get the latest transaction
            latest_transaction = self.record.iloc[-1]

            # Check if the stock is already in the position
            if latest_transaction['stock_name'] in self.position['stock_name'].values:
                # Update the existing position
                idx = self.position[self.position['stock_name'] == latest_transaction['stock_name']].index
                self.position.loc[idx, 'amount'] += latest_transaction['amount']
                self.position.loc[idx, 'price'] += latest_transaction['price']
                self.position.loc[idx, 'avg_price'] = self.position.loc[idx, 'price'] / self.position.loc[idx, 'amount']
                self.position = self.position[self.position['amount'] >0]
            else:
                new_row = pd.Series({'stock_name': latest_transaction['stock_name'],
                    'amount': latest_transaction['amount'],
                    'price': latest_transaction['price'],
                    'avg_price': latest_transaction['avg_price']})
                self.position.loc[len(self.position)] = new_row 
            self.__save_to_csv()
                
                


    def buy_stock(self, date, stock, amount, price):#買股票

        '''
        to buy stock and update the balance and position
        input : stock(Stock) , amount(int)
        output : bool   
        '''

        if price * amount * (1 + self.handling_fee) > self.balance:
            print("交割餘額不足")
        
        else:
            print("預計交割金額", price * amount * (1 + self.handling_fee))
            if  self.auto == True:
                self.balance -= price * amount * (1 + self.handling_fee)
                new_row = pd.Series({'date': date,
                    'stock_name': stock,
                    'amount': amount,
                    'price': price * amount * (1 + self.handling_fee),
                    'avg_price': price * (1 + self.handling_fee),})
                self.record.loc[len(self.record)] = new_row
                self.__update()

    def sell_stock(self, date, stock, amount, price):

        '''
        to sell stock and update the balance and position
        input : stock(Stock) , amount(int)
        output : bool
        '''

        stock_positions = self.position[self.position['stock_name'] == stock]
        if stock_positions['amount'].sum() < amount:
            print("股票不足")
            return False
        print("預計獲利金額", price * amount * (1 - self.handling_fee - 0.003))
        
        if self.auto == True :
            self.balance += price * amount * (1 + self.handling_fee) * 0.997
            self.record.loc[len(self.record)] = pd.Series(
                {
                    'date': date,
                    'stock_name': stock,
                    'amount': -amount,
                    'price': -price * (1 - self.handling_fee - 0.003) * amount,
                    'avg_price':  price * (1 - self.handling_fee - 0.003) 
                },
                )
            self.__update()
            

c:\Users\User\anaconda3\lib\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


初始化user

In [2]:
Jolin = User("Jolin")
Jolin.auto = True

In [3]:
Jolin.position

,stock_name,amount,price,avg_price


In [4]:
Jolin.record

,date,stock_name,amount,price,avg_price


# buy stock


In [8]:
'''
產生購買訊號
input : date ->datetime , stock -> string , amount -> int , price -> int
'''
date = dt.datetime(2022,1,4)
stock = "2330.TW"
taiwan_2330 = yf.Ticker(stock)

try:
  data = taiwan_2330.history(start = date,end = date+dt.timedelta(days=1))
  buy_price = data["Close"][0]
  Jolin.buy_stock(date,stock,1000,buy_price)
except Exception as e:
  print("今天可能沒有開盤!!")
  
print("目前資金:",Jolin.balance)
Jolin.position

預計交割金額 629484.1784683227
目前資金: 370515.82153167727


,stock_name,amount,price,avg_price
0,2330.TW,1000,629484.178468,629.484178


# sell stock

In [9]:
'''
產生賣出訊號
input : date ->datetime , stock -> string , amount -> int , price -> int
'''
date = dt.datetime(2024,1,4)
stock = "2330.TW"
taiwan_2330 = yf.Ticker(stock)
try:
  data = taiwan_2330.history(start = date,end = date+dt.timedelta(days=1))
  sell_price = data["Close"][0]
  Jolin.sell_stock(date,stock,1000,sell_price)
except Exception as e:
  print("今天可能沒有開盤!!")
print("目前資金:",Jolin.balance)
Jolin.position



預計獲利金額 577433.5
目前資金: 949599.8420316773


,stock_name,amount,price,avg_price
